### Clustering Algorithm
##### Customer Segmentation based on marketing_campaign.csv

### Data Analysis

In [2]:
import pandas as pd

file_path = 'marketing_campaign.csv'
data = pd.read_csv(file_path, sep='\t')
num_records, num_features = data.shape
print("First 10 rows:")
print(data.head(10))
null_values = data.isnull().sum()
print("\nNumber of nulls:")
print(null_values)
print("\nTypes of data:")
print(data.dtypes)


First 10 rows:
     ID  Year_Birth   Education Marital_Status   Income  Kidhome  Teenhome  \
0  5524        1957  Graduation         Single  58138.0        0         0   
1  2174        1954  Graduation         Single  46344.0        1         1   
2  4141        1965  Graduation       Together  71613.0        0         0   
3  6182        1984  Graduation       Together  26646.0        1         0   
4  5324        1981         PhD        Married  58293.0        1         0   
5  7446        1967      Master       Together  62513.0        0         1   
6   965        1971  Graduation       Divorced  55635.0        0         1   
7  6177        1985         PhD        Married  33454.0        1         0   
8  4855        1974         PhD       Together  30351.0        1         0   
9  5899        1950         PhD       Together   5648.0        1         1   

  Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
0  04-09-2012       58       635  ...               

#### Data Processing

In [3]:
# Cleaning Data
# Drop null values
data_cleaned = data.dropna().copy()
# New attribute [Age] = 2023 - Year_Birth 
data_cleaned['Age'] = 2023 - data_cleaned['Year_Birth']
# New attribute [TotalAmountSpent]
data_cleaned['TotalAmountSpent'] = data_cleaned['MntFishProducts'] + data_cleaned['MntFruits'] + data_cleaned['MntGoldProds'] + data_cleaned['MntSweetProducts'] + data_cleaned['MntMeatProducts'] + data_cleaned['MntWines']
# Drop symbolic values
data_processed = data_cleaned.drop(['ID', 'Education', 'Marital_Status', 'Dt_Customer', 'Year_Birth'], axis=1)
# Show
num_records_processed, num_features_processed = data_processed.shape
print(f"Number of records: {num_records_processed}, number of features: {num_features_processed}")
null_values = data_processed.isnull().sum().sum()
print(f"Number of null values: {null_values}")


Number of records: 2216, number of features: 26
Number of null values: 0


In [5]:
from sklearn.preprocessing import StandardScaler

# Data standardization
scaler = StandardScaler()
data_normalized_standard = scaler.fit_transform(data_processed)